In [1]:
import pandas as pd

In [8]:
params = pd.read_csv('data.csv')
params.head(2)

,name,build_power,speed,e_storage,m_storage,e_production,m_production,m_cost,e_cost,bp300bt,bp80bt,max_count
0,initial,NaN,NaN,500.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,com,300.0,NaN,500.0,500.0,25.0,2.0,2700.0,26000.0,NaN,NaN,NaN


In [23]:
data = {
        'entity': ['com', 'mex', 'solar', 'wind','con', 'mc', 'nano', 'solar_adv'],
        'count': [1, 3, 2, 3, 1, 0, 0, 0]
       }

df = pd.DataFrame(data)
df.to_csv('initial_t1.csv')
df.head(2)

,entity,count
0,com,1
1,mex,3


In [24]:
# Merge the two dataframes based on the 'entity' and 'name' columns
merged_df = pd.merge(df, params[[
    'name', 
    'build_power', 
    'e_production',
    'e_cost',
    'm_cost'
]], left_on='entity', right_on='name', how='left')
merged_df['bp_amount']=merged_df.build_power*merged_df['count']
merged_df

,entity,count,name,build_power,e_production,e_cost,m_cost,bp_amount
0,com,1,com,300.0,25.0,26000.0,2700.0,300.0
1,mex,3,mex,NaN,-3.0,500.0,50.0,NaN
2,solar,2,solar,NaN,20.0,0.0,150.0,NaN
3,wind,3,wind,NaN,8.0,175.0,45.0,NaN
4,con,1,con,80.0,NaN,1750.0,125.0,80.0
5,mc,0,mc,NaN,-70.0,1250.0,1.0,NaN
6,nano,0,nano,200.0,NaN,3200.0,210.0,0.0
7,solar_adv,0,solar_adv,NaN,75.0,4000.0,370.0,NaN


In [25]:
merged_df.bp_amount.sum()

380.0

In [32]:
steps = {
        'e_amount': [272],
        'm_amount': [248],
        'e_production': [99],
        'm_production': [8],
        'order': [''],
        'time':[0]
       }
steps = pd.DataFrame(steps)
steps

,e_amount,m_amount,e_production,m_production,order,time
0,272,248,99,8,,0


In [33]:
candidates = ['solar','wind','solar_adv','mc']
step = 0
# Check hypotesis
for candidate in candidates:
    row = merged_df[merged_df.entity==candidate].iloc()[0]
    step_row = steps.iloc()[step]
    e_cost = row.e_cost
    m_cost = row.m_cost
    e_amount = step_row.e_amount
    m_amount = step_row.m_amount
    if e_cost > e_amount or m_cost > m_amount:
        print('no:', candidate, e_cost, m_cost)
        continue
    e_production = row.e_production
    print(candidate, e_cost, m_cost, e_production)

solar 0.0 150.0 20.0
wind 175.0 45.0 8.0
no: solar_adv 4000.0 370.0
no: mc 1250.0 1.0


In [ ]:
new_row = {
    'e_amount': 300,
    'm_amount': 275,
    'e_production': 120,
    'm_production': 10,
    'order': 'sol',
    'time': 1
}
steps.loc[len(steps)] = new_row